In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import os
import glob
from pandas import DataFrame
from collections import Counter
from konlpy.tag import Twitter

In [2]:
# 파일 합치는 곳 (폴더 제목 입력)
youtube_channel = input("유투브 채널 이름 입력하시오.")
path =  "GAME/" + str(youtube_channel) + "/comment"
path2 = "GAME/" + str(youtube_channel) + '\\result.csv'
arg_youtube = str(youtube_channel) + "*"


allFile_list = glob.glob(os.path.join(path,arg_youtube))
print(allFile_list)
allData = []
for file in allFile_list:
    df = pd.read_csv(file)
    allData.append(df)
    
dataCombine = pd.concat(allData,axis =0, ignore_index = True)

dataCombine.to_csv(path2, index = False)
cdPath = "GAME/" + str(youtube_channel) + '/result.csv'
comment_data = pd.read_csv(cdPath)

유투브 채널 이름 입력하시오.감스트GAMST
['GAME/감스트GAMST/comment\\감스트GAMST_comment_1.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_10.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_11.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_12.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_13.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_14.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_15.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_16.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_17.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_18.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_19.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_2.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_20.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_3.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_4.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_5.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_6.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_7.csv', 'GAME/감스트GAMST/comment\\감스트GAMST_comment_8.csv', 'GAME/감스트GAMST/comment\\감스트GAMST

In [3]:
comment_data

,Unnamed: 0,youtube_id,comment,like_num
0,0,감스트GAMST,영상 좋아요 꼭 좀 부탁드립니다!!!!,743개
1,1,최민우,"7:31 ""They are TALMO""",10개
2,2,김두한의오마니와아바이,안녕하세요 인직이형!!! 4:00에서 타란튤라 보낸 중딩입니다!!! 제가 보낸 타란...,23개
3,3,박희산,다 아이네 아이유 아이린 아이즈원 아기잉직ㅋㅋㅋㅋ,186개
4,4,ShowJean,안녕하세요.. 형 저 처음에 보낸 김세진이예요.... \n스타대회도 재미있게 보고...,195개
...,...,...,...,...
2949,155,슈슈공장,역시 gta할때가 가장웃겨 ㅋㅋ,1개
2950,156,민또베,요번영상 탱구 ㅈㄴ웃기네 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,1개
2951,157,lIlIIIlIll,ㅋㅋㅋㅋㅋㅋ 이거 분명 그 유튜버분께서 길게만든다,0개
2952,158,악수정은이와,0:27 여자때리..직,1개


In [4]:
#이모티콘 제거
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

#분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

In [5]:
comment_list = []
for i in range(len(comment_data)):
    comment_list.append(comment_data['comment'].iloc[i])

In [6]:
comment_result = []

for i in comment_list:
    tokens = re.sub(emoji_pattern,"",i)
    tokens = re.sub(han,"",tokens)
    comment_result.append(tokens)

comment_result = pd.DataFrame(comment_result, columns=["comment"])

In [7]:
comment_result

,comment
0,영상 좋아요 꼭 좀 부탁드립니다
1,7:31 They are TALMO
2,안녕하세요 인직이형 4:00에서 타란튤라 보낸 중딩입니다 제가 보낸 타란튤라는 아직...
3,다 아이네 아이유 아이린 아이즈원 아기잉직
4,안녕하세요 형 저 처음에 보낸 김세진이예요 스타대회도 재미있게 보고 택배컨텐츠를 ...
...,...
2949,역시 gta할때가 가장웃겨
2950,요번영상 탱구 웃기네
2951,이거 분명 그 유튜버분께서 길게만든다
2952,0:27 여자때리직


In [8]:
#한글 형태소 분석기인 konlpy사용 
#이때 konlpy에는 hannanum, kommoran, kkma, twitter이 있는데 해당 분석에서는 kkma사용
from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.tag import Twitter
#apply lambda사용

In [9]:
### 추출된 단어가 무슨 형태인지 파악
twitter = Twitter()
sentences_tag = []

for sentence in comment_list:
    morph = twitter.pos(sentence)
    sentences_tag.append(morph)
print(morph)
sentences_tag

C:\Anaconda\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


[('8:58', 'Number'), ('ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 'KoreanParticle'), ('개', 'Noun'), ('웃겨', 'Verb')]


[[('영상', 'Noun'),
  ('좋아요', 'Adjective'),
  ('꼭', 'Noun'),
  ('좀', 'Noun'),
  ('부탁드립니다', 'Adjective'),
  ('!!!!', 'Punctuation')],
 [('7:31', 'Number'),
  ('"', 'Punctuation'),
  ('They', 'Alpha'),
  ('are', 'Alpha'),
  ('TALMO', 'Alpha'),
  ('"', 'Punctuation')],
 [('안녕하세요', 'Adjective'),
  ('인직', 'Noun'),
  ('이형', 'Noun'),
  ('!!!', 'Punctuation'),
  ('4:00', 'Number'),
  ('에서', 'Josa'),
  ('타란튤라', 'Noun'),
  ('보낸', 'Verb'),
  ('중딩', 'Noun'),
  ('입니다', 'Adjective'),
  ('!!!', 'Punctuation'),
  ('제', 'Noun'),
  ('가', 'Josa'),
  ('보낸', 'Verb'),
  ('타란튤라', 'Noun'),
  ('는', 'Josa'),
  ('아직', 'Adverb'),
  ('알', 'Noun'),
  ('에서', 'Josa'),
  ('나온지', 'Verb'),
  ('얼마', 'Noun'),
  ('안된', 'Adjective'),
  ('개체', 'Noun'),
  ('입니다', 'Adjective'),
  ('.(', 'Punctuation'),
  ('먹이는', 'Verb'),
  ('제', 'Noun'),
  ('가', 'Josa'),
  ('작은', 'Adjective'),
  ('애벌레', 'Noun'),
  ('를', 'Josa'),
  ('통', 'Noun'),
  ('에', 'Josa'),
  ('조금', 'Noun'),
  ('넣어', 'Verb'),
  ('놓았습니다', 'Verb'),
  ('!!!)', 'Punctuation'),


In [10]:
# stop_words 가져오기

f = pd.read_csv('불용어사전(한글자).csv',encoding='euc-kr')
stop_words = list(f)
stop_words.extend(['거','저','내','더','용','수','안'])
stop_words.remove('Unnamed: 69')

In [26]:
def get_noun(comment_txt):
    
    twitter = Twitter()
    noun_list = []
    
    if len(comment_txt) > 0:
        twitter = twitter.pos(comment_txt, norm = True, stem = True)
        for word, tag in twitter:
            if tag in ['Noun']:
                if word not in stop_words:
                    noun_list.append(word)
                    
    return noun_list

In [27]:
def get_adj(comment_txt):

    twitter = Twitter()
    adj_list = []
    
    if len(comment_txt) > 0:
        twitter = twitter.pos(comment_txt, norm = True, stem = True)
        for word, tag in twitter:
            if tag in ['Adjective']:
                if word not in stop_words:
                    adj_list.append(word)
                    
    return adj_list

In [28]:
def get_verb(comment_txt):

    twitter = Twitter()
    verb_list = []
    
    if len(comment_txt) > 0:
        twitter = twitter.pos(comment_txt, norm = True, stem = True)
        for word, tag in twitter:
            if tag in ['Verb']:
                if word not in stop_words:
                    verb_list.append(word)
    
    return verb_list

In [29]:
comment_result['noun'] = comment_result['comment'].apply(lambda x: get_noun(x))
comment_result['adj'] = comment_result['comment'].apply(lambda x: get_adj(x))
comment_result['verb'] = comment_result['comment'].apply(lambda x: get_verb(x))

In [30]:
comment_result

,comment,noun,adj,verb
0,영상 좋아요 꼭 좀 부탁드립니다,"[영상, 꼭]","[좋다, 부탁드리다]",[]
1,7:31 They are TALMO,[],[],[]
2,안녕하세요 인직이형 4:00에서 타란튤라 보낸 중딩입니다 제가 보낸 타란튤라는 아직...,"[인직, 이형, 타란튤라, 중딩, 타란튤라, 알, 얼마, 개체, 애벌레, 통, 조금...","[안녕하다, 이다, 안되다, 이다, 작다, 좋아하다, 불편하다, 같다, 나쁘다, 심...","[보내다, 보내다, 나오다, 먹이다, 넣다, 놓다, 읽다, 보내다, 드리다, 하다,..."
3,다 아이네 아이유 아이린 아이즈원 아기잉직,"[아이, 아이유, 아이린, 아이즈, 아기, 잉직]",[],[]
4,안녕하세요 형 저 처음에 보낸 김세진이예요 스타대회도 재미있게 보고 택배컨텐츠를 ...,"[형, 처음, 김세진, 스타, 대회, 보고, 택배, 컨텐츠, 알, 일찍, 택배, 제...","[안녕하다, 재미있다, 놀래다, 아니다, 재미있다, 재미있다, 같다, 재미있다, 속...","[보내다, 되어다, 보내다, 되어다, 보내다, 하다, 해주다, 보내다, 모르다, 보..."
...,...,...,...,...
2949,역시 gta할때가 가장웃겨,"[역시, 가장]",[],"[하다, 웃기다]"
2950,요번영상 탱구 웃기네,"[번영, 웃기]",[],[]
2951,이거 분명 그 유튜버분께서 길게만든다,"[분명, 유튜버분]",[길다],[만들다]
2952,0:27 여자때리직,"[여자, 직]",[],[때리다]


In [31]:
pd_Path = 'GAME/' + str(youtube_channel) +'/token_result.csv'
comment_result.to_csv(pd_Path, index = False, encoding = 'utf-8-sig')